## Question 1: **Answer**

In [1]:
import pandas as pd
import sqlite3

In [ ]:
def add_data_to_database(input_data):
    try:
        data = pd.read_csv(input_data)
        # Creating a database
        engine = sqlite3.connect('music_business.db')
        data.to_sql('music', con=engine, index = False)
    except Exception as e:
        print(f"Error: {e}")

data = "/content/music_store.csv"
add_data_to_database(data)

In [3]:
# Loading SQL extension
%load_ext sql

In [4]:
#  Connecting to the database
%sql sqlite:///music_business.db

In [5]:
# Default to duplicated style
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [6]:
%%sql
SELECT * FROM music
LIMIT 5;

 * sqlite:///music_business.db
Done.


customer_id,customer_name,contact_number,purchase_date,product_type,product_name,quantity,price
1,Alice Smith,123-456-7890,2023-01-05,Instruments,Guitar,13.0,200.0
2,Bob Johnson,987-654-3210,2023-10-07,Instruments,Keyboard,20.0,300.0
3,Charlie Brown,567-890-1234,2023-02-07,Accessories,Strings,18.0,50.0
4,David Williams,789-012-3456,2023-02-09,Accessories,Pedals,21.0,80.0
5,Emily Jones,345-678-9012,2023-03-09,Music Books,Theory Book,19.0,100.0


## Question 2: **Answer**

In [7]:
%%sql
SELECT
    customer_name,
    product_type
FROM music
WHERE
    customer_name = 'Charlie Brown' AND
    (product_type = 'Instruments' OR product_type = 'Accessories')

 * sqlite:///music_business.db
Done.


customer_name,product_type
Charlie Brown,Accessories


## Question 3: **Answer**

In [8]:
%%sql
-- CTE to calculate the maximum quantity
WITH max_quantity AS(
    SELECT
    MAX(quantity) AS max_quatity
    FROM music
)
-- Main query to return name, quantity and day
SELECT
    customer_name,
    quantity,
    STRFTIME('%d', purchase_date) AS day_of_month
FROM music
WHERE quantity = (SELECT * FROM max_quantity);

 * sqlite:///music_business.db
Done.


customer_name,quantity,day_of_month
David Williams,21.0,09
Peter Jones,21.0,02


## Question 4: **Answer**

In [9]:
%%sql
SELECT
    SUM(quantity * price) AS total_sales_value,
    SUM(quantity) AS total_quantity,
    -- Calculate sales value for high-value products
    SUM(CASE WHEN price > 100 THEN quantity * price ELSE 0 END)
    AS high_price_sales_value,
    -- Calculate quantity value for high-value products
    SUM(CASE WHEN price > 100 THEN quantity ELSE 0 END)
    AS high_price_quantity
FROM music
WHERE purchase_date BETWEEN '2023-10-21' AND '2023-12-02';

 * sqlite:///music_business.db
Done.


total_sales_value,total_quantity,high_price_sales_value,high_price_quantity
12865.0,111.0,10800.0,18.0


 ## Question 5: **Answer**

In [10]:
%%sql
-- CTE calculating total sales per product
WITH total_sales AS (
    SELECT
        product_name,
        SUM(quantity * price) AS total
    FROM music
    WHERE product_name LIKE 'Guitar%' OR product_name LIKE '%Guitar'
    GROUP BY product_name)

--Main Query to return revenue and percentage
SELECT
    SUM(total) AS total_revenue_guitar,
    ROUND(sum(total)/(SELECT SUM(quantity * price) FROM music), 2)
    AS revenue_percentage
FROM total_sales;

 * sqlite:///music_business.db
Done.


total_revenue_guitar,revenue_percentage
8850.0,0.19


## Altenative method

In [11]:
%%sql
SELECT
-- Calculating the guitar products revenue
    SUM(CASE
        WHEN product_name LIKE 'Guitar%' OR product_name LIKE '%Guitar'
        THEN quantity * price
        ELSE 0
    END) AS total_revenue_guitar,
    -- Calculating the percentage contribution of guitar revenue
    ROUND(SUM(CASE
        WHEN product_name LIKE 'Guitar%' OR product_name LIKE '%Guitar'
        THEN quantity * price
        ELSE 0
    END) / SUM(quantity * price), 2) AS percentage
FROM music;

 * sqlite:///music_business.db
Done.


total_revenue_guitar,percentage
8850.0,0.19


In [12]:
# %%sql
# DROP TABLE music;